## Downloading data from NIFC server

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import lxml
import cchardet

In [ ]:
IR_URL = "https://ftp.wildfire.gov/public/incident_specific_data/calif_n/2012%20FEDERAL_Incidents/CA-KNF-5659_FortComplex/IR/"
GIS_URL = "https://ftp.wildfire.gov/public/incident_specific_data/calif_n/2012%20FEDERAL_Incidents/CA-KNF-5659_FortComplex/incident_data/"

In [ ]:
soup = BeautifulSoup(requests.get(IR_URL).content)
paths = [IR_URL + a.get('href') for a in soup.find_all('a')[5:]]
paths

In [ ]:
soup = BeautifulSoup(requests.get(paths[0]).content)
files = [paths[0] + a.get('href') for a in soup.find_all('a')[5:]]
files

In [ ]:
requests.get(files[0])

### Full Scraping Algorithm

In [ ]:
def download(source, target):
    print('downloading {0} to {1}'.format(source, target))
    
    if source.lower().endswith('products/'):
        print('...skipping')
        return
    
    try:
        if source[-1] != '/':
            if not os.path.isfile(target):
                if source.lower().endswith(('.pdf', '.docx', '.doc', '.tif', '.png', '.jpg', '.xlsx')):
                    print('...skipping')
                    return

                os.makedirs(os.path.dirname(target), exist_ok=True)

                with open(target, 'wb') as f:
                    f.write(session.get(source).content)
            else:
                print('...present')
            return

        soup = BeautifulSoup(session.get(source).content, 'lxml')

        for a in soup.find_all('a')[5:]:
            path = a.get('href')
            download(source + path, target + path)
    except:
        return

In [ ]:
BASE_URL = './data/'
ROOT_URL = 'https://ftp.wildfire.gov/public/incident_specific_data/'

session = requests.Session()

download(ROOT_URL, BASE_URL)

## Filtering of downloaded data (after unarchiving)

In [ ]:
import sys
import os
import subprocess

In [ ]:
k = 1
for fire_dir in os.listdir('download/all_fire_data/'):
    if fire_dir != '.DS_Store':
        num = 1_000_000
        i = 0
        for data_dir in os.listdir(f'download/all_fire_data/{fire_dir}'):
            if data_dir != '.DS_Store':
                num = min(num, len(next(os.walk(f'download/all_fire_data/{fire_dir}/{data_dir}'))[1]))
                i += 1
        if i == 2 and num > 10 and num < 1_000_000:
            print(k, fire_dir)
            subprocess.run(['cp', '-r', f'download/all_fire_data/{fire_dir}', f'download/fire_data/{fire_dir}'])
            k += 1

In [ ]:
k = 1
for fire_dir in os.listdir('download/fire_data/'):
#     fire_dir = 'CA-AEU-024918_Butte'
    print(fire_dir)
    if fire_dir != '.DS_Store':
        for fire_day in os.listdir(f'download/fire_data/{fire_dir}/GIS'):
            print(f'download/fire_data/{fire_dir}/GIS/{fire_day}')
            if fire_day != '.DS_Store':
                res = subprocess.run(['find', f'download/fire_data/{fire_dir}/GIS/{fire_day}', '-type', 'f', '-name', '*.shp'])
#                 print(res.stdout)
                if res.stdout is None:
#                     print('\'echo $(dirname {})/$(basename {} .gdb) {}\'')
                    print('hi1')
                    res = subprocess.run(['find', f'download/fire_data/{fire_dir}/GIS/{fire_day}', '-name', '*.gdb',
                                    '-exec', 'sh', '-c', 'ogr2ogr -f \"ESRI Shapefile\" $(dirname {})/shapefiles-$(basename {} .gdb) {}', ';'])
                    print('hi2')
#                     print(res.stdout)

In [ ]:
subprocess.run(['find', f'download/fire_data/', '-name', '*.gdb', '-exec', 'sh', '-c', 'ogr2ogr -f \"ESRI Shapefile\" $(dirname {})/shapefiles-$(basename {} .gdb) {}', ';'])



